<center><h1>Scikit Pipelines ❇️❇️❇️</h1></center>
</br>
<center><h4><em>Work in Progress</em></h4></center>
<center><em>Feel free to comment, suggest or anything :)</em></center>

# Simple Setup

In [ ]:
%reset -sf

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
# Patch Xeon Intel OneAPI Scikit accelerator
!pip install scikit-learn-intelex
from sklearnex import patch_sklearn
patch_sklearn()

# Data

In [ ]:
# Reading data

from pandas import read_csv

train = read_csv('/kaggle/input/tabular-playground-series-mar-2022/train.csv')
test = read_csv('/kaggle/input/tabular-playground-series-mar-2022/test.csv')
sample_subm = read_csv('/kaggle/input/tabular-playground-series-mar-2022/sample_submission.csv')

In [ ]:
# Dates format

from pandas import to_datetime

def encode_time(df):
    df = df.copy()
    df['time'] = to_datetime(df['time'])
    return df

train = encode_time(train)
test = encode_time(test)

In [ ]:
# Encode direction column

from sklearn.preprocessing import OrdinalEncoder

def encode_direction(df):
    df = df.copy()
    oe = OrdinalEncoder()
    _ = oe.fit(df[['direction']])
    df['direction_c'] = oe.transform(df[['direction']])
    df = df.drop(['row_id', 'direction'], axis=1)
    return df

train = encode_direction(train)
test = encode_direction(test)

In [ ]:
# Inspecting Roads

'x =>', train['x'].unique()
'y =>', train['y'].unique()
'direction =>', train['direction_c'].unique()

from pandas import crosstab

crosstab(train['x'], train['direction_c'], values='congestion', aggfunc='count')
crosstab(train['y'], train['direction_c'], values='congestion', aggfunc='count')

# There are some combinations not present, therefore 
# total number of roads is not 3 * 4 * 8 = 96, rather the aforementioned 65

# Feature Engineering

In [ ]:
# Feature Engineering

def feat_eng(df):
    df = df.copy()
    #df['year'] = df.time.dt.year
    df['month'] = df.time.dt.month
    df['dayofweek'] = df.time.dt.dayofweek
    df['day'] = df.time.dt.day
    df['hour'] = df.time.dt.hour
    df['minute'] = df.time.dt.minute
    df = df.drop('time', axis=1)
    return df
    
train = feat_eng(train)
test = feat_eng(test)

In [ ]:
[(col, len(train[col].unique())) for col in train]

# Training

In [ ]:
# Split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train.drop('congestion', axis=1), 
                                                    train['congestion'], 
                                                    test_size=0.1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# Pipe

from numpy import linspace

from sklearn.pipeline import make_pipeline, make_union
from sklearn.compose import make_column_transformer

from sklearn.preprocessing import StandardScaler, RobustScaler, PolynomialFeatures, FunctionTransformer

from sklearn.linear_model import Ridge, Lars, LinearRegression, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor, IsolationForest, BaggingRegressor, VotingRegressor, AdaBoostRegressor, StackingRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

from lightgbm import LGBMRegressor

from sklearn.model_selection import RandomizedSearchCV, StratifiedShuffleSplit, StratifiedGroupKFold, GroupShuffleSplit

union = make_union(FunctionTransformer(lambda x: x), 
                   PolynomialFeatures(interaction_only=True))

pipe = make_pipeline(
    union, 
    StandardScaler(), 
    HistGradientBoostingRegressor(loss='absolute_error', 
                                  max_iter=150,
                                  random_state=16) ) 

pipe.get_params()

splitter = GroupShuffleSplit(n_splits=7, test_size=0.1, random_state=16)

pipe = RandomizedSearchCV(pipe, { 
    'histgradientboostingregressor__l2_regularization': linspace(0,1), 
    'histgradientboostingregressor__learning_rate': linspace(0.01, 0.5), 
    'histgradientboostingregressor__max_iter': linspace(2,100).astype(int), 
    'histgradientboostingregressor__max_leaf_nodes': linspace(5,50).astype(int), 
    'histgradientboostingregressor__min_samples_leaf': linspace(5,50).astype(int), }, 
                          n_iter=20, 
                          cv=splitter.split(X_train, y_train, X_train['hour']),
                          n_jobs=-1,
                          scoring='neg_mean_absolute_error', 
                          verbose=True) 

pipe.fit(X_train, y_train)

In [ ]:
# Eval

mean_absolute_error(y_test, pipe.predict(X_test))

# Submission

In [ ]:
# Subm

sample_subm['congestion'] = pipe.predict(test)
sample_subm.to_csv('submission.csv', index=False)

# If you find it useful, please upvote 😀